**Apuntes útiles pandas**<br>
- https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python#question3
- https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe
- https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/

Se ha establecido un dataframe con la siguiente estructura

|generación|Puerta#1 x| Puerta#1 y|Puerta#2 x| Puerta#2 y|Puerta#3 x| Puerta#3 y|Puerta#4 x| Puerta#4 y|Puerta#5 x| Puerta#5 y|
|-|-|-|-|-|-|-|-|-|-|-|
|..|..|..|..|..|..|..|..|..|..|..|

Donde cada fila corresponderá a un individuo

In [82]:
# https://pynetlogo.readthedocs.io/en/latest/_docs/pynetlogo.html
# https://pynetlogo.readthedocs.io/en/latest/example1.html
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import platform
import pyNetLogo
import random
gui=False
current_gen = 0 #contador de la generación actual

print("inicializando, espere...")
try:
    sistema=platform.system()
    if(sistema=='Windows'):
        netlogo = pyNetLogo.NetLogoLink(gui=False)
    elif(sistema=='Linux'):
        netlogo = pyNetLogo.NetLogoLink(gui=gui,netlogo_home='NetLogo 6.0.4/')
    else:
        netlogo = pyNetLogo.NetLogoLink(gui=gui)
    print("Listo!")
except:
    print("Ocurrio un problema al levantar netlogo!")

inicializando, espere...
Listo!


In [30]:
df=pd.DataFrame(columns=['gen','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5']) #dataframe que representará a los individuos existentes
ww=netlogo.report("world-width") #obtener el número de columnas de la grilla
wh=netlogo.report("world-height") #obtener el número de filas de la grilla
plan_file = "conference.plan" #archivo plano 
world=list() #representación del mundo en una lista multidimensional (filasXcolumnas)

wall_val=64 #valor de muralla
inner_val=0 #valor espacio interno edificio
outer_Val=2 #valor espacio externo edificio
door_val=45 #valor para las puertas


In [31]:
#abrir el archivo de plano para inicializar la representación de listas multidimensionales
with open(plan_file) as file:
    row=list()
    counter=0
    for i in file:
        patch=i.replace('\n','').split(' ')
        if(counter%ww !=0):
            row.append(int(patch[2]))
        else:
            row=list()
            world.append(row)
        counter = counter + 1
        
#cargar el programa en netlogo
netlogo.load_model(r'escape4_v6.nlogo')
#cargar el plano en netlogo
netlogo.command("load-plan-file")

In [32]:
#simulate recibe la lista ordenada de coordenadas de puertas, realiza la simulación correspondiente y retorna los ticks que ha demorado
#La distribución de puertas debe estar previamente validada, en caso de que las puertas no sean accesibles, la simulación podría quedar atascada en un while infinito
def simulate(doors):
    netlogo.command("set door-data [ [{0} {1}] [{2} {3}] [{4} {5}] [{6} {7}] [{8} {9}] ]".format(*doors))
    netlogo.command("show-plan")
    netlogo.command("generate-population")
    while(netlogo.report("count turtles")>0):
        netlogo.command("go")
    return(netlogo.report("ticks"))

#retorna una lista ordenadas de puertas seleccionadas al azar (válidas)
def pick_random_doors(n):
    #falta implementar
    doors = [0,0,1,1,2,2,3,3,4,4]
    return  doors

#reinicia el dataframe y crea la primera generación, recibe la cantidad de individuos a generar
def genesis(n):
    global df
    df=pd.DataFrame(columns=['gen','x1','y1','x2','y2','x3','y3','x4','y4','x5','y5']) #dataframe que representará a los individuos existentes
    for i in range(n):
        doors_loc=pick_random_doors(5)
        df.loc[df.shape[0]]=[current_gen] + doors_loc    
def catch_the_good_doors(): #Obtener las coordenadas x e y de todas las puertas que nos sirven
    x=netlogo.report("listax")
    y=netlogo.report("listay")
    arr=[]
    for i in range(len(x)):
        arr.append([x[i],y[i]])
    return arr

In [6]:
#ejemplo de simulación para conference.plan
doors=[20,50,40,10,80,15,95,30,60,100]
simulate(doors)


122.0

In [7]:
genesis(10)
df

,gen,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5
0,0,0,0,1,1,2,2,3,3,4,4
1,0,0,0,1,1,2,2,3,3,4,4
2,0,0,0,1,1,2,2,3,3,4,4
3,0,0,0,1,1,2,2,3,3,4,4
4,0,0,0,1,1,2,2,3,3,4,4
5,0,0,0,1,1,2,2,3,3,4,4
6,0,0,0,1,1,2,2,3,3,4,4
7,0,0,0,1,1,2,2,3,3,4,4
8,0,0,0,1,1,2,2,3,3,4,4
9,0,0,0,1,1,2,2,3,3,4,4


In [8]:
l=[1,11,2,2,3,3,4,4,5,5]
s="set door-data [ [{0} {1}] [{2} {3}] [{4} {5}] [{6} {7}] [{8} {9}] ]".format(*l)
s
#ask patches with [pcolor = 64 and (count other patches in-radius 1 with [pcolor = black]) = 1 and (count other patches in-radius 1 with [pcolor = 2]) = 1 ][set pcolor yellow]

'set door-data [ [1 11] [2 2] [3 3] [4 4] [5 5] ]'

In [17]:
netlogo.kill_workspace()

In [33]:
tuplas=catch_the_good_doors() 

In [34]:
largo=len(tuplas)
# tuplas

# Creación Generación 0

In [35]:
pop_size=20
coord_puertas_Gen0=[]
indice_Puertas=[]
for i in range(pop_size):
    coordenadas_puertas_Individuo=[]
    indice_puertas_Individuo=[]
    for x in range(5):
        pos=random.randint(0,largo-1)
        indice_puertas_Individuo.append(pos)
        coordenadas_puertas_Individuo.append(tuplas[pos][0])
        coordenadas_puertas_Individuo.append(tuplas[pos][1])
    indice_Puertas.append(indice_puertas_Individuo)
    coord_puertas_Gen0.append(coordenadas_puertas_Individuo)
#     print(puertasIndividuo)
#   simulate(puertasIndividuo)

In [39]:
# coord_puertas_Gen0
indice_Puertas

[[99, 228, 193, 91, 335],
 [98, 296, 83, 329, 320],
 [260, 317, 25, 85, 15],
 [34, 59, 324, 197, 37],
 [84, 207, 333, 25, 250],
 [189, 282, 111, 2, 165],
 [339, 121, 168, 209, 45],
 [245, 158, 56, 243, 344],
 [189, 275, 134, 337, 110],
 [195, 312, 49, 276, 86],
 [267, 296, 346, 2, 120],
 [210, 121, 271, 243, 0],
 [164, 206, 66, 303, 348],
 [109, 134, 286, 286, 16],
 [113, 209, 185, 134, 301],
 [275, 297, 288, 168, 260],
 [289, 23, 156, 31, 161],
 [76, 256, 264, 276, 317],
 [194, 225, 52, 62, 42],
 [208, 127, 125, 349, 236]]

In [37]:
# import time
iteraciones=5
ticks=[]
# t0 = time.time()
for i in range(len(coord_puertas_Gen0)):
    auxTicks=[]
    for itera in range(iteraciones):
        auxTicks.append(simulate(coord_puertas_Gen0[i]))
#     print(sum(auxTicks)/float(iteraciones))
    ticks.append(sum(auxTicks)/float(iteraciones))
ticks
# print("%s  Segundos "%(time.time()-t0))

[109.4,
 109.8,
 134.0,
 130.0,
 93.6,
 131.2,
 117.2,
 83.8,
 144.4,
 111.0,
 111.6,
 110.6,
 114.8,
 193.4,
 124.2,
 81.6,
 116.6,
 116.2,
 100.6,
 160.4]

# Crossover

In [80]:

criterio=100
thaBest=[]
for i in range (individuos):
    if (ticks[i]<=criterio):
        thaBest.append(indice_Puertas[i])
if(len(thaBest)%2==0):
    print("apareence mierda")
else:
    print("uno quedo solo :'c")

uno quedo solo :'c


# Mutacion

In [234]:
prob_mutacion=0.3
for individuo in indice_Puertas:
    if(random.random()<=prob_mutacion):
        pos=random.randint(0,4)
        nuevaPuerta=random.randint(0,largo-1)
        if(indice_Puertas[individuo][pos]!=nuevaPuerta):
            indice_Puertas[individuo][pos]=nuevaPuerta
#terminar mutación, corregir posibles errores

228